In [ ]:
import sys
sys.path.insert(0, '../..') 

import torch
import torch.sparse

from src.sae_function import topk_sae, topk_sae_fwd_sparse_coo, topk_sae_fwd_sparse_csr, topk_sae_sparse_fused
from src.utils.compare import compare_reconstruct, compare_reconstruct_grad, compare_latency
from src.config import SAETestConfig

device='cuda'
dtype=torch.float32

In [ ]:
config = SAETestConfig(
    batch_size=4096,
    d_model=2048,
    expansion_factor=8,
    topk=128,
    device=device,
    dtype=dtype,
)

In [ ]:
compare_reconstruct_grad(
    sae_fwd_1=topk_sae,
    sae_fwd_2=topk_sae_fwd_sparse_csr,
    config=config,
    atol = 1e-5,
    rtol = 1e-5,
)

reconstruct is Same: True
max=1.91e-06, mean=1.07e-07
W_E.grad    is Same: True
max=4.66e-08, mean=4.69e-09
W_D.grad    is Same: True
max=1.49e-08, mean=1.23e-09
b_E.grad    is Same: True
max=3.35e-08, mean=4.91e-09
b_D.grad    is Same: True
max=5.96e-08, mean=7.24e-09


In [11]:
compare_latency(
    sae_fwd_1=topk_sae_fwd_sparse_csr,
    sae_fwd_2=topk_sae_sparse_fused,
    config=config
)

100%|██████████| 20/20 [00:00<00:00, 35.52it/s]

name                    sae_fwd_1    sae_fwd_2      speedup
---------------------------------------------------------------------------
fwd                       7.824ms      7.300ms        1.07x
compute_loss              0.091ms      0.094ms        0.97x
backward                  7.885ms      4.055ms        1.94x
---------------------------------------------------------------------------
total                    15.800ms     11.449ms        1.38x


In [5]:
compare_reconstruct_grad(
    sae_fwd_1=topk_sae,
    sae_fwd_2=topk_sae_sparse_fused,
    config=config
)

reconstruct is Same: True
max=2.15e-06, mean=1.07e-07
W_E.grad    is Same: True
max=4.47e-08, mean=4.55e-09
W_D.grad    is Same: True
max=1.49e-08, mean=1.22e-09
b_E.grad    is Same: True
max=3.35e-08, mean=5.42e-09
b_D.grad    is Same: True
max=8.94e-08, mean=7.17e-09
